# How to get codes from #1326 and #1327 to work with Qibo in Jupyter notebook?

- IQM uses {identity, R, CZ} as native gates. R in Braket == Phase RX in Qiskit & Qibo. Phase RX gate is not implemented in main Qibo branch yet, it can be found in this PR: (https://github.com/qiboteam/qibo/pull/1326)
- For ZNE testing, the code that does global unitary folding can be found in this PR: (https://github.com/qiboteam/qibo/pull/1327)
- As a side note, how do we get these codes in the 2 PRs to work in a notebook?
    - In virtual environment, do pip install qibo, and maybe jupyter notebook
    - Go to /path/to/your/environment/lib/python-3.11/site-packages/qibo
    - Replace /gates/gates.py, /gates/abstract.py, and /backends/npmatrices.py with those found in PR#1326
    - Replace /models/error_mitigation.py* with those found in PR#1327. (*May need to remove "from qibo.backends import _check_backend_and_local_state" if an error pops up)

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.jp-Cell { margin-left: -0% !important; }</style>"))
display(HTML("<style>.jp-Cell { width: 100% !important; }</style>"))

# Import

In [2]:
# some_file.py
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.append('/Users/matt/Documents/GitHub/qibo-cloud-backends_AWS/src/qibo_cloud_backends/')

from aws_client import BraketClientBackend

# Latest aws_client.py

In [7]:
from qibo.backends import NumpyBackend
from qibo.config import raise_error
from qibo.result import MeasurementOutcomes, QuantumState
from qibo import gates
from qibo import Circuit as QiboCircuit
from qibo.transpiler.pipeline import Passes, assert_transpiling
from qibo.transpiler.optimizer import Preprocessing
from qibo.transpiler.router import ShortestPaths
from qibo.transpiler.unroller import Unroller, NativeGates
from qibo.transpiler.placer import Random

import re
import importlib.util
import sys
import networkx as nx

# Import Qiskit packages for transpiler
from qiskit import transpile
from qiskit import QuantumCircuit
from qiskit.transpiler import CouplingMap
from qiskit.circuit.random import random_circuit
from qiskit import qasm2

from braket.aws import AwsDevice, AwsQuantumTask
from braket.circuits import Gate, observables
from braket.circuits import Circuit as BraketCircuit
from braket.devices import Devices, LocalSimulator

_QASM_BRAKET_GATES = {
    "id": "i",
    "cx": "cnot",
    "sx": "v",
    "sxdg": "vi",
    "sdg": "si",
    "tdg": "ti",
    "u3": "U",
}

class BraketClientBackend(NumpyBackend):
    """Backend for the remote execution of AWS circuits on the AWS backends.

    Args:
        device (str): The ARN of the Braket device. Defaults to Braket's statevector LocalSimulator, LocalSimulator("default").
                      Other devices are Braket's density matrix simulator, LocalSimulator("braket_dm"), or any other
                      QPUs.
        verbatim_circuit (bool): If `True`, wrap the Braket circuit in a verbatim box to run it on the QPU
                                 without any transpilation. Defaults to `False`.
        transpilation (bool): If `True`, check for two additional arguments: native_gates and coupling_map.
        native_gates: - For qibo transpiler:   (list, qibo.gates): e.g. [gates.I, gates.RZ, gates.SX, gates.X, gates.ECR]
                      - For qiskit transpiler: (list, str):        e.g. ['ecr', 'i', 'rz', 'sx', 'x']
        coupling_map (list, list): E.g. [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]
    """

    def __init__(self, device=None, verbatim_circuit=False, transpilation=False, native_gates=None, coupling_map=None):
        super().__init__()
        
        self.verbatim_circuit = verbatim_circuit

        self.transpilation = transpilation
        if transpilation:
            print('Transpile!')
            if coupling_map is None:
                raise_error(ValueError, "Expected qubit_map. E.g. qubit_map = [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]")
            else:
                self.coupling_map = coupling_map
            if native_gates is None:
                raise_error(ValueError, "Expected native gates for transpilation. E.g. native_gates = [gates.I, gates.RZ, gates.SX, gates.X, gates.ECR]")
            else:
                self.native_gates = native_gates

        
        if device is None:
            self.device = LocalSimulator("default")
            # self.device = LocalSimulator("braket_dm")
        else:
            self.device = device
        self.name = "aws"

    def remove_qelib1_inc(self, qasm_string):
        """To remove the 'includes qe1lib.inc' from the OpenQASM string.

        Args: 
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.
        Returns:
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.
        """
        
        # Remove the "include "qelib1.inc";\n" line
        modified_code = re.sub(r'include\s+"qelib1.inc";\n', '', qasm_string)
        return modified_code
    
    def qasm_convert_gates(self, qasm_code):
        """To replace the notation for certain gates in OpenQAS.

        Args: 
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.
        Returns:
            qasm_code (OpenQASM circuit, str): circuit given in the OpenQASM format.
        """
        
        lines = qasm_code.split('\n')
        modified_code = ""
        for line in lines:
            for key in _QASM_BRAKET_GATES:
                if key in line:
                    line = line.replace(key, _QASM_BRAKET_GATES[key])
                    break
            modified_code += line + '\n'
        return modified_code

    def custom_connectivity(self, coupling_map):
        """Converts a coupling map given in list form to a networkx graph. Returns networkx graph.
    
        Args:
            coupling_map (list): E.g. [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]
        Returns:
            graph (networkx graph): graph
        """
        
        graph = nx.Graph()
        for connection in coupling_map:
            q1, q2 = connection
            graph.add_edge(q1, q2)
        return graph

    def transpile_qibo_to_qibo_with_qibo(self, circuit):
        """Transpiles a Qibo circuit with a specific topology specified by connectivity and custom_native_gates.
            There is no option for optimization_level like Qiskit. Therefore, no gates will be collapsed.
            Returns a Qibo circuit.
    
        Args:
            circuit (qibo.models.Circuit): Circuit to transpile
        Returns:
            transpiled_circuit (qibo.models.Circuit): Transpiled circuit.
            final_layout (dict): dict of connectivity?
        """
        
        # Define custom passes as a list
        custom_passes = []
        
        # Preprocessing adds qubits in the original circuit to match the number of qubits in the chip
        custom_passes.append(Preprocessing(connectivity=self.custom_connectivity(self.coupling_map)))
    
        # Placement step
        custom_passes.append(Random(connectivity=self.custom_connectivity(self.coupling_map))) 
    
        # Routing step
        custom_passes.append(ShortestPaths(connectivity=self.custom_connectivity(self.coupling_map)))
    
        # custom_native_gates = [gates.I, gates.RZ, gates.SX, gates.X, gates.ECR]
        custom_native_gates = [gates.I, gates.Z, gates.U3, gates.CZ]
        custom_passes.append(Unroller(native_gates=NativeGates.from_gatelist(self.native_gates))) # Gate decomposition ste
    
        custom_pipeline = Passes(custom_passes, 
                                 connectivity=self.custom_connectivity(self.coupling_map),
                                 native_gates=NativeGates.from_gatelist(self.native_gates)) 
                                # native_gates=NativeGates.default()
    
        transpiled_circuit, final_layout = custom_pipeline(circuit)
    
        return transpiled_circuit, final_layout

    def transpile_qasm_circuit_with_qiskit(self, circuit_qasm, native_gates, custom_coupling_map, optimization_level=1):
        """Transpiles a circuit given in OpenQASM format using Qiskit's transpiler.
            Returns a transpiled circuit given in OpenQASM format.

        Args:
            circuit_qasm (OpenQASM circuit, str): circuit given in the OpenQASM format.
            native_gates (list, str): A list of strings representing the native gates of the QPU.
                e.g. native_gates = ['ecr', 'i', 'rz', 'sx', 'x']
            custom_coupling_map (list, list): A list containing lists representing the connectivity of the qubits.
                e.g. custom_coupling_map. E.g. [[0, 1], [1, 2], [2, 3]]
            optimization_level (int): Optimization level for Qiskit's transpiler. Range is from 0 to 3. Defaults to 1.

        Returns:
            transpiled_circuit_qasm (OpenQASM circuit, str): Transpiled circuit in the OpenQASM format.
        """

        self.native_gates = native_gates
        self.custom_coupling_map = custom_coupling_map

        if optimization_level < 0 or optimization_level > 3:
            raise_error(ValueError, "Optimization_level is between 0 to 3.")
        if custom_coupling_map is None:
            raise_error(ValueError, "Expected custom_coupling_map. E.g. custom_coupling_map = [[0, 1], [1, 2], [2, 3]]")
        if native_gates is None:
            raise_error(ValueError, "Expected native gates for transpilation. E.g. native_gates = ['ecr', 'i', 'rz', 'sx', 'x']")

        # Convert to Qiskit circuit for transpilation
        circuit_qiskit = QuantumCircuit.from_qasm_str(circuit_qasm)
        transpiled_circuit = transpile(circuit_qiskit,
                                       basis_gates = self.native_gates,
                                       optimization_level = self.optimization_level,
                                       coupling_map = self.custom_coupling_map)
        transpiled_circuit_qasm = qasm2.dumps(transpiled_circuit) # Convert back to qasm.
        return transpiled_circuit_qasm

    def transpile_qibo_to_braket_with_qiskit(self, circuit_qibo, optimization_level=1):
        """Transpiles a Qibo circuit using Qiskit's transpiler. Returns a Braket circuit.
    
        Args:
            circuit_qibo (qibo.models.Circuit): Qibo circuit to transpile.
            native_gates (list, str): A list of strings representing the native gates of the QPU.
                e.g. native_gates = ['ecr', 'i', 'rz', 'sx', 'x']
            custom_coupling_map (list, list): A list containing lists representing the connectivity of the qubits.
                e.g. custom_coupling_map. E.g. [[0, 1], [1, 2], [2, 3]]
            optimization_level (int): Optimization level for Qiskit's transpiler. Range is from 0 to 3. Defaults to 1.
    
        Returns:
            transpiled_circuit_braket (braket.circuits.Circuit): Braket circuit to that has been transpiled.
        """

        print('Transpile qibo to braket using qiskit')
        
        if optimization_level < 0 or optimization_level > 3:
            raise_error(ValueError, "Optimization_level is between 0 to 3.")
        else:
            self.optimization_level = optimization_level
        if custom_coupling_map is None:
            raise_error(ValueError, "Expected custom_coupling_map. E.g. custom_coupling_map = [[0, 1], [1, 2], [2, 3]]")
        if native_gates is None:
            raise_error(ValueError, "Expected native gates for transpilation. E.g. native_gates = ['ecr', 'i', 'rz', 'sx', 'x']")

        circuit_qasm = circuit_qibo.to_qasm()
        circuit_qiskit = QuantumCircuit.from_qasm_str(circuit_qasm)
        transpiled_circuit = transpile(circuit_qiskit,
                                       basis_gates = self.native_gates,
                                       optimization_level = self.optimization_level,
                                       coupling_map = self.coupling_map)
        transpiled_circuit_qasm = qasm2.dumps(transpiled_circuit) # Convert back to qasm.
        transpiled_circuit_qasm = self.remove_qelib1_inc(transpiled_circuit_qasm)
        transpiled_circuit_qasm = self.qasm_convert_gates(transpiled_circuit_qasm)
        braket_circuit = BraketCircuit.from_ir(transpiled_circuit_qasm)
        return braket_circuit

    def transpile_qibo_to_qibo_with_qiskit(self, circuit_qibo, optimization_level=1):
        """Transpiles a Qibo circuit using Qiskit's transpiler. Returns a Qibo circuit.
    
        Args:
            circuit_qibo (qibo.models.Circuit): Qibo circuit to transpile.
            native_gates (list, str): A list of strings representing the native gates of the QPU.
                e.g. native_gates = ['ecr', 'i', 'rz', 'sx', 'x']
            custom_coupling_map (list, list): A list containing lists representing the connectivity of the qubits.
                e.g. custom_coupling_map. E.g. [[0, 1], [1, 2], [2, 3]]
            optimization_level (int): Optimization level for Qiskit's transpiler. Range is from 0 to 3. Defaults to 1.
    
        Returns:
            transpiled_circuit_qibo (qibo.models.Circuit): Qibo circuit that has been transpiled.
        """

        print('Transpile qibo to qibo using qiskit transpiler')
        
        if optimization_level < 0 or optimization_level > 3:
            raise_error(ValueError, "Optimization_level is between 0 to 3.")
        else:
            self.optimization_level = optimization_level
        if self.coupling_map is None:
            raise_error(ValueError, "Expected custom_coupling_map. E.g. custom_coupling_map = [[0, 1], [1, 2], [2, 3]]")
        if self.native_gates is None:
            raise_error(ValueError, "Expected native gates for transpilation. E.g. native_gates = ['ecr', 'i', 'rz', 'sx', 'x']")

        circuit_qasm = circuit_qibo.to_qasm()
        circuit_qiskit = QuantumCircuit.from_qasm_str(circuit_qasm)
        transpiled_circuit = transpile(circuit_qiskit,
                                       basis_gates = self.native_gates,
                                       optimization_level = self.optimization_level,
                                       coupling_map = self.coupling_map)
        transpiled_circuit_qasm = qasm2.dumps(transpiled_circuit) # Convert back to qasm.
        transpiled_circuit_qibo = QiboCircuit.from_qasm(transpiled_circuit_qasm)
        return transpiled_circuit_qibo

    def execute_circuit(self,
                        circuit,
                        nshots=1000,
                        **kwargs):
        """Executes a Qibo circuit on an AWS Braket device. The device defaults to the LocalSimulator().
            
        Args:
            circuit (qibo.models.Circuit): circuit to execute on the Braket device.
            nshots (int): Total number of shots.
        Returns:
            Measurement outcomes (qibo.measurement.MeasurementOutcomes): The outcome of the circuit execution.
        """
        
        measurements = circuit.measurements
        if not measurements:
            raise_error(RuntimeError, "No measurement found in the provided circuit.")
        nqubits = circuit.nqubits
        circuit_qasm = circuit.to_qasm()
        circuit_qasm = self.remove_qelib1_inc(circuit_qasm)
        circuit_qasm = self.qasm_convert_gates(circuit_qasm)
        braket_circuit = BraketCircuit.from_ir(circuit_qasm)

        if self.verbatim_circuit:
            braket_circuit = BraketCircuit().add_verbatim_box(braket_circuit)
        result = self.device.run(braket_circuit, shots=nshots).result()
        samples = result.measurements

        return MeasurementOutcomes(
            measurements=measurements, backend=self, samples=samples, nshots=nshots
        )


In [3]:
# circuit_qibo = Circuit(2)
# circuit_qibo.add(gates.RX(0, np.pi/7))
# circuit_qibo.add(gates.CNOT(1,0))
# circuit_qibo.add(gates.M(0))
# circuit_qibo.add(gates.M(1))

# circuit_qasm = circuit_qibo.to_qasm()
# circuit_qiskit = QuantumCircuit.from_qasm_str(circuit_qasm)

# # from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# # pm = generate_preset_pass_manager(backend=AWS, optimization_level=1)
# # isa_qc = pm.run(qc)

# 00. Extracting native gates from device

In [ ]:
from braket.aws import AwsDevice
from braket.devices import Devices, LocalSimulator

device = LocalSimulator()
# device = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'
# device = 'arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1'

client = AwsDevice(device)
print(client.properties.paradigm.nativeGateSet)
# print(client.properties)


# 0. Running a circuit on AWS

In [ ]:
from qibo import Circuit, gates
import numpy as np
from braket.aws import AwsDevice
from braket.devices import Devices, LocalSimulator

circuit_qibo = Circuit(2)
circuit_qibo.add(gates.RX(0, np.pi/7))
circuit_qibo.add(gates.CNOT(1,0))
circuit_qibo.add(gates.M(0))
circuit_qibo.add(gates.M(1))
print('>> Original circuit')
print(circuit_qibo.draw())

# SV1 = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
# device = AwsDevice(SV1)
device = LocalSimulator("braket_dm")
AWS = BraketClientBackend(device = device)

out = AWS.execute_circuit(circuit_qibo, nshots=1000)
print(out.frequencies())

# 1. Instantiate a quantum circuit

In [3]:
from qibo import Circuit, gates
import numpy as np
from qibo.backends import NumpyBackend
from qibo.models.error_mitigation import get_noisy_circuit, ZNE
# from qibo.models.error_mitigation import ZNE
from qibo.symbols import Z
from qibo.hamiltonians import SymbolicHamiltonian
from qibo.backends import GlobalBackend
from qibo.noise import DepolarizingError, NoiseModel, ReadoutError

circuit_qibo = Circuit(2)
circuit_qibo.add(gates.RX(0, np.pi/7))
circuit_qibo.add(gates.CNOT(1,0))
# circuit_qibo.add(gates.GPI(0, np.pi/3))# circuit_qibo.add(gates.M(0))
# circuit_qibo.add(gates.GPI2(1, np.pi/3))# circuit_qibo.add(gates.M(0))
# circuit_qibo.add(gates.MS(0, 1, np.pi/3, np.pi/4, np.pi/5))# circuit_qibo.add(gates.M(0))
# circuit_qibo.add(gates.U3(0, 1, np.pi/3, np.pi/4, np.pi/5))
circuit_qibo.add(gates.M(0))
circuit_qibo.add(gates.M(1))
print('>> Original circuit')
print(circuit_qibo.draw())


>> Original circuit
q0: ─RX─X─M─
q1: ────o─M─


In [80]:
from qibo.config import raise_error
def get_noisy_circuit(circuit, num_insertions: int, global_unitary_folding=True, insertion_gate=None):
    """Standalone function to generate the noisy circuit with the inverse gate pairs insertions.

    Args:
        circuit (:class:`qibo.models.circuit.Circuit`): circuit to modify.
        num_insertions (int): number of insertion gate pairs / global unitary folds to add.
        global_unitary_folding (bool): Default True. Global unitary folding is carried out by default.
            Global unitary folding takes precedence over insertion_gate.
        insertion_gate (str, optional): gate to be used in the insertion.
            If ``"RX"``, the gate used is :math:``RX(\\pi / 2)``.
            Otherwise, the other gate is ``"CNOT"``.

    Returns:
        :class:`qibo.models.Circuit`: circuit with the inserted gate pairs or with global folding.
    """

    if global_unitary_folding: # pragma: no cover
        circuit_no_meas = circuit.__class__(**circuit.init_kwargs)
        circuit_meas = circuit.__class__(**circuit.init_kwargs)
        for gate in circuit.queue:
            if gate.name != 'measure':
                circuit_no_meas.add(gate)
            else:
                circuit_meas.add(gate)
        
        noisy_circuit = circuit.__class__(**circuit.init_kwargs)
        noisy_circuit = circuit_no_meas
        for _ in range(num_insertions):
            noisy_circuit += circuit_no_meas.invert() + circuit_no_meas

        noisy_circuit += circuit_meas
        
    else:
        if insertion_gate is None or insertion_gate not in ("CNOT", "RX"):  # pragma: no cover
                raise_error(
                    ValueError,
                    "Invalid insertion gate specification. Please select between 'CNOT' and 'RX'.",
                    )
        if insertion_gate == "CNOT" and circuit.nqubits < 2:  # pragma: no cover
            raise_error(
                ValueError,
                "Provide a circuit with at least 2 qubits when using the 'CNOT' insertion gate. "
                + "Alternatively, try with the 'RX' insertion gate instead.",
            )
    
        i_gate = gates.CNOT if insertion_gate == "CNOT" else gates.RX
    
        theta = np.pi / 2
        noisy_circuit = circuit.__class__(**circuit.init_kwargs)
    
        for gate in circuit.queue:
            noisy_circuit.add(gate)
    
            if isinstance(gate, i_gate):
                if insertion_gate == "CNOT":
                    control = gate.control_qubits[0]
                    target = gate.target_qubits[0]
                    for _ in range(num_insertions):
                        noisy_circuit.add(gates.CNOT(control, target))
                        noisy_circuit.add(gates.CNOT(control, target))
                elif gate.init_kwargs["theta"] == theta:
                    qubit = gate.qubits[0]
                    for _ in range(num_insertions):
                        noisy_circuit.add(gates.RX(qubit, theta=theta))
                        noisy_circuit.add(gates.RX(qubit, theta=-theta))
    
    return noisy_circuit

# 2. noisy_circuit = get_noisy_circuit(...)

In [7]:
noisy_circuit = get_noisy_circuit(circuit_qibo, num_insertions = 3, global_unitary_folding=True, insertion_gate='CNOT')
print(noisy_circuit.draw())


q0: ─RX─X─X─RX─RX─X─X─RX─RX─X─X─RX─RX─X─M─
q1: ────o─o───────o─o───────o─o───────o─M─


# 3. Transpile. Use BraketClientBackend.transpile_qibo_to_qibo.
# Input native_gates, custom_coupling_map

### a. qiskit_ionq package for native gates

In [61]:
from qiskit_ionq import GPIGate, GPI2Gate, MSGate
from qiskit import QuantumCircuit
from qiskit import qasm2
from qiskit.quantum_info import Operator

cc = QuantumCircuit(3)

cc.append(GPIGate(1.234), [0])
# cc.append(GPI2Gate(1.234), [0])
# cc.append(MSGate(1, 2, 3), [0, 2])

# print(qasm2.dumps(cc))
# operator = Operator(cc)
# print(operator.data)

In [ ]:
cc.draw('mpl')

In [ ]:
print(GPIGate)

### b. Apply native gates

In [53]:
custom_coupling_map = [[0, 1], [0, 7], [1, 2], [2, 3], [4, 3], [4, 5], [6, 5], [7, 6]]
# custom_coupling_map = [[0, 1], [0, 2]]
# native_gates = ['GPIGate', 'GPI2Gate', 'MSGate']
# native_gates = ["i", "rz", "sx", "x", "ecr"]
native_gates = ["i", "rz", "sx", "x", "cz"]


# qiskit.circuit.library.GMS(num_qubits, theta)

# device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/sv1')
device = LocalSimulator("braket_dm")
# device = 'arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1'
AWS = BraketClientBackend(device = device, verbatim_circuit=False, transpilation=True, native_gates=native_gates, coupling_map=custom_coupling_map)

try:
    transpiled_circuit = AWS.transpile_qibo_to_qibo_with_qiskit(noisy_circuit)
    
    print('\n>> Original circuit')
    print(circuit_qibo.draw())
    
    print('\n>> Transpiled circuit')
    print(transpiled_circuit.draw())

except Exception as e:
    print(f"Error! {e}")


Transpile!
Transpile qibo to qibo using qiskit transpiler

>> Original circuit
q0: ─RX─X─M─
q1: ────o─M─

>> Transpiled circuit
q0: ─RZ─SX─RZ─o─RZ─SX─RZ─M─
q1: ──────────Z─M──────────
q2: ───────────────────────
q3: ───────────────────────
q4: ───────────────────────
q5: ───────────────────────
q6: ───────────────────────
q7: ───────────────────────


# 4. Set backend = BraketClientBackend

### a. Check that both original and transpiled circuit run properly on GlobalBackend()

In [54]:
backend = GlobalBackend()

res_original = backend.execute_circuit(circuit_qibo, nshots=1000)
print(res_original.frequencies())

res_transpiled = backend.execute_circuit(transpiled_circuit, nshots=1000)
print(res_transpiled.frequencies())

Counter({'00': 949, '10': 51})
Counter({'00': 951, '10': 49})


### b. Now try AWS

In [68]:
backend = AWS

res_original = backend.execute_circuit(circuit_qibo, nshots=1000)
print(res_original.frequencies())

res_transpiled = backend.execute_circuit(transpiled_circuit, nshots=1000)
print(res_transpiled.frequencies())


Counter({'00': 948, '10': 52})
Counter({'00': 927, '10': 73})


/Users/matt/Envs/AWS_latest_qibo_qiskit_braket/lib/python3.11/site-packages/braket/default_simulator/simulator.py:328: UserWarning: You are running a noise-free circuit on the density matrix simulator. Consider running this circuit on the state vector simulator: LocalSimulator("default") for a better user experience.
  warnings.warn(


### Manually compute expectation value

In [64]:
Zmatrix = gates.Z(0).matrix()
ZZmatrix = np.kron(Zmatrix, Zmatrix)
ZZdiag = np.real(np.diag(ZZmatrix))

dict = {}
for ii in range(2**circuit_qibo.nqubits):
    dict[bin(ii)[2:].zfill(circuit_qibo.nqubits)] = 0
for key, val in res_original.frequencies().items():
    dict[key] = val/sum(res_original.frequencies().values())

manual_expectation_val = 0
for key, val in dict.items():
    manual_expectation_val += val * int(ZZdiag[int(key, 2)])

print('Manually computing expectation value of ZZ operators = %.6f' %manual_expectation_val)


Manually computing expectation value of ZZ operators = 0.904000


__Scenarios - failure:__

1. This circuit will fail because `q0` is not used. Transpiled circuit needs to be contiguous.
``` Python
q0: ──────────────
q1: ─U3─U3─Z─U3─M─
q2: ───────o─M──── 
```
Error message: `ValueError: Non-contiguous qubit indices supplied; qubit indices in a circuit must be contiguous.` (AWS device error)

2. This circuit will fail because of some mismatch in dimensions.
``` Python
q0: ────────────────U3─U3─Z─U3─M─
q1: ─U3─Z─U3─o─U3─Z─U3────o─M────
q2: ────o─U3─Z─U3─o──────────────
q3: ─────────────────────────────
```
Error message: `ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 3)` (Error translating back to Qibo) <-- <mark style="background-color: yellow"> **Solution**: Update Qibo >= 0.2.6 </mark>

__Scenario - success:__

1. This scenario passes. The coupling map can't get any simpler: `custom_coupling_map = [[0, 1]]`
``` Python
q0: ─U3─U3─Z─U3─M─
q1: ───────o─M────
```

# 5. Run ZNE with backend = BraketClientBackend

In [65]:
num_meas = 0
for data in transpiled_circuit.raw['queue']:
    if data['name'] == 'measure':
        num_meas += 1 
print(num_meas)

2


In [66]:
ZNE_circuit = transpiled_circuit # circuit_qibo
quantum_state = ZNE_circuit.execute().state()

# obs = np.prod([Z(i) for i in range(ZNE_circuit.nqubits-1)])
obs = np.prod([Z(i) for i in range(num_meas)])
print(obs)
obs_exact = SymbolicHamiltonian(obs, nqubits=ZNE_circuit.nqubits, backend=backend)
print(obs_exact)
obs = SymbolicHamiltonian(obs, backend=backend)
print(obs)

exact = obs_exact.expectation(quantum_state)
state = backend.execute_circuit(ZNE_circuit, nshots=10000)

print(state)

Z0*Z1
-0.97493j|00000000> + (-0.22252-0j)|10000000>


In [67]:
estimate = ZNE(
    circuit=ZNE_circuit,
    observable=obs,
    noise_levels=np.array(range(5)),
    noise_model=None,
    nshots=100000,
    solve_for_gammas=False,
    insertion_gate="CNOT",
    readout=None,
    backend=backend,
)
print('ZNE estimated expectation value = %.6f' %estimate)

ZNE estimated expectation value = 0.899042


# Running on a Braket device?
### - Costs: DM1 & SV1: \\$0.075 per minute, TN1: \\$0.275 per minute
### - Run time: 

In [ ]:
- Transpile to Ion Q's native gates
- Change to Ion Q
- Run on Ion Q

In [ ]:
- Transpile to IQM's native gates
- Change to IQM
- Run on IQM